<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 4



### 6.1 Softmax Prediction
* To classify more than two classes



#### 1D



Three classes:
* `y = 0`
* `y = 1`
* `y = 2`



In [ ]:
import functools
import pathlib
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn
import torchvision.datasets
import torchvision.transforms



In [ ]:
x = torch.linspace(-2.0, 2.0, 41)
y = (x > -1)*1.0 + (x > 1)*1.0



In [ ]:
def plot_softmax_data_00(x, y, ax=None):

  if ax is None:
    ax = plt.gca()

  x0 = x[y<0.5]
  x1 = x[(0.5<y) & (y<1.5)]
  x2 = x[1.5 < y]

  ax.plot(x0.numpy(), np.zeros(x0.shape), 'b.', label='y=0')
  ax.plot(x1.numpy(), np.zeros(x1.shape), 'r.', label='y=1')
  ax.plot(x2.numpy(), np.zeros(x2.shape), 'g.', label='y=2')

  ax.set_xlabel('x')
  ax.legend(loc=0)
  ax.grid(True)

  return ax


plot_softmax_data_00(x, y)



Three functions
* $z_0(x) = w_0 x + b_0$
* $z_1(x) = w_1 x + b_1$
* $z_2(x) = w_2 x + b_2$



Classification
* $\hat {\theta} = \underset{i}{argmax}z_i(x)$



In [ ]:
def plot_softmax_lines_01(x, ax=None):

  if ax is None:
    ax = plt.gca()

  w0, b0 = -1.0, -0.1
  w1, b1 = 0.1, 1.0
  w2, b2 = 1.1, -0.1

  y0 = w0 * x.numpy() + b0
  y1 = w1 * x.numpy() + b1
  y2 = w2 * x.numpy() + b2

  ax.plot(x.numpy(), y0, 'b-', label='$z_0$')
  ax.plot(x.numpy(), y1, 'r-', label='$z_1$')
  ax.plot(x.numpy(), y2, 'g-', label='$z_2$')

  ax.set_ylim(bottom=-0.05, top=1.9)

  ax.legend(loc=0)

  return ax


ax = plot_softmax_data_00(x, y)
ax = plot_softmax_lines_01(x, ax=ax)



#### 2D
* MNIST dataset
* See classification results as vectors



* take a look at the MNIST data
* Following cell would only work in Google Colab



In [ ]:
@functools.lru_cache
def read_sample_data():

  sample_data_path = pathlib.Path('sample_data')
  assert sample_data_path.exists()
  assert sample_data_path.is_dir()

  mnist_path = sample_data_path / 'mnist_train_small.csv'
  assert mnist_path.exists()
  assert mnist_path.is_file()

  df_mnist = pd.read_csv(mnist_path)

  y = df_mnist.iloc[:, 0].astype(int)
  x = df_mnist.iloc[:, 1:].astype(int)

  return {'x': x, 'y': y}


@functools.lru_cache
def get_x_of_y(y:int, d_xy=None):
  if d_xy is None:
    d_xy = read_sample_data()

  return d_xy['x'].loc[y == d_xy['y'], :]


def get_x_i(y:int, i:int=None, x_of_y=None, shape=(28, 28)):
  if x_of_y is None:
    x_of_y = get_x_of_y(y)

  if i is None:
    i = random.randint(0, x_of_y.shape[0])

  return np.array(x_of_y.iloc[i, :]).reshape(*shape)


if 'google.colab' in str(get_ipython()):
  # TODO : if not Google Colab, read from torchvision
  random.seed()
  _, axs = plt.subplots(3, 3)
  for y, ax in enumerate(axs.flatten()):
    x_i = get_x_i(y)
    ax.imshow(x_i);



### 6.2 Softmax Function



Custom module



In [ ]:
class Softmax(torch.nn.Module):
  def __init__(self, in_size:int, out_size:int):
    super(Softmax, self).__init__()
    self.linear = torch.nn.Linear(in_size, out_size)

  def forward(self, x):
    return self.linear(x)



In [ ]:
model = Softmax(3, 3)

X = torch.tensor([
    [3.0, 1.0, 0.1],
    [0.0, 2.0, 1.0],
    [1.0, 2.0, 3.0],
])

z = model(X)
_, yhat = z.max(1)

print(z)

print(yhat)



### 6.3 Softmax PyTorch
* Load Data
* Create Model
* Train Model
* View Results



#### Load Data



In [ ]:
import torch
import torch.nn
import torchvision.transforms
import torchvision.datasets



In [ ]:
@functools.lru_cache
def load_MNIST_torchvision():
  '''
  [[image, class], ...]
  '''

  train_dataset = torchvision.datasets.MNIST(
      root='./data',
      train=True,
      download=True,
      transform=torchvision.transforms.ToTensor(),
  )

  test_dataset = torchvision.datasets.MNIST(
      root='./data',
      train=False,
      download=True,
      transform=torchvision.transforms.ToTensor(),
  )

  return {
      'train': train_dataset,
      'validation': test_dataset,
  }



In [ ]:
d_mnist = load_MNIST_torchvision()
x0, y0 = random.choice(d_mnist['train'])
ax = plt.imshow(np.squeeze(x0))
plt.title(f'y = {y0}');



#### Create Model



In [ ]:
input_dim = 28*28
output_dim = 10

model = Softmax(input_dim, output_dim)
z = model(x0.flatten())



In [ ]:
print('W:', list(model.parameters())[0].size())
print('b:', list(model.parameters())[1].size())



In [ ]:
def PlotParameters(model):
  _, axs = plt.subplots(2, 5)
  w_3d = list(model.parameters())[0]
  for k, (w_2d, ax) in enumerate(zip(w_3d, axs.flatten())):
    ax.imshow(w_2d.detach().numpy().reshape(28, 28))
    ax.set_xlabel(f'Weights : {k}')


PlotParameters(model)



In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

n_epochs = 100



In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset=d_mnist['train'], batch_size=100,
)

validation_loader = torch.utils.data.DataLoader(
    dataset=d_mnist['validation'], batch_size=5000,
)



#### Train Model



In [ ]:
import os
import time

start_time = time.time()

N_test = len(d_mnist['validation'])
accuracy_list = []

if os.environ.get('CI', False):
  n_epochs = 1

for epoch in range(n_epochs):
  for x, y in train_loader:
    optimizer.zero_grad()
    z = model(x.view(-1, 28*28))
    loss = criterion(z, y)
    loss.backward()
    optimizer.step()

  correct = 0

  for x_test, y_test in validation_loader:
    z = model(x_test.view(-1, 28*28))
    _, yhat = torch.max(z.data, 1)

    correct += (yhat == y_test).sum().item()
  accuracy = correct / N_test
  accuracy_list.append(accuracy)

  print(f"{epoch:4d} {accuracy:.6} {(time.time() - start_time):16.4f}")



In [ ]:
plt.stem(accuracy_list)



#### View Results



In [ ]:
PlotParameters(model)



## Shallow Neural Networks



### 7.1 Neural Networks in One Dimension



In [ ]:
import torch
import torch.nn as nn



In [ ]:
class Net(torch.nn.Module):
  def __init__(self, *argv, **kwarg):
    super(Net, self).__init__()

    self.layers = []
    for n_in, n_out in (argv[:-1], argv[1:]):
      self.layers.append(torch.nn.Linear(n_in, n_out))

  def forward(self, x):

    for layer in self.layers:
      x = torch.sigmoid(layer(x))

    return x



In [ ]:
model = Net(1, 2, 1)

x = torch.tensor([0.0])
yhat = model(x)

yhat



### 7.2 Neural Networks More Hidden Neurons



### 7.3 Neural Networks with Multiple Dimensional Input



### 7.4 Multi-Class Neural Networks



### 7.5 Backpropagation



### 7.6 Activation Function

